In [22]:
import sys
sys.path.append('../common')
import utils
import importlib
importlib.reload(utils)
from utils import IOHandler as IO

import time
import os
import datetime
import numpy as np
import pynput.keyboard as kb

from PIL import Image
from enum import Enum
from collections import defaultdict
from functools import lru_cache


io = IO(offset=(880, 558), game_dims=(1600,1200), verbose=True)

In [ ]:
class Tile(Enum):
    ANCHOR_BOX = "anchor-box"
    BACKGROUND = "background"
    BED = "bed"
    BENCH = "bench"
    BLANK = "blank"
    BLENDER = "blender"
    BLUE_DRAGON = "blue-dragon"
    BLUE_FISH = "blue-fish"
    CHAIR = "chair"
    CHEST = "chest"
    CLOWNFISH = "clownfish"
    COIN = "coin"
    COLUMN = "column"
    CRAB = "crab"
    FOSSIL = "fossil"
    FRIDGE = "fridge"
    GOLDFISH = "goldfish"
    GREEN_SEAHORSE = "green-seahorse"
    LIONFISH = "lionfish"
    LOCKED = "locked"
    MAGENTA_FISH = "magenta-fish"
    ORANGE_FISH = "orange-fish"
    OVEN = "oven"
    PINK_FISH = "pink-fish"
    PURPLE_DRAGON = "purple-dragon"
    PURPLE_FISH = "purple-fish"
    ROUND_VASE = "round-vase"
    SEAL = "seal"
    SHELL = "shell"
    STARFISH = "starfish"
    THRONE = "throne"
    TRIANGLE_VASE = "triangle-vase"
    TURTLE = "turtle"
    UMBRELLA = "umbrella"
    WHALE = "whale"
    WHEEL_TABLE = "wheel-table"
    YELLOW_FISH = "yellow-fish"
    YELLOW_SEAHORSE = "yellow-seahorse"

class Dir(Enum):
    LEFT = "⥢"
    RIGHT = "⥤"
    UP = "⥣"
    DOWN = "⥥"

class Screen(Enum):
    BONUS = "bonus"
    OUT_OF_MOVES = "out_of_moves"
    GAME_OVER = "game_over"
    CONGRATS = "congrats"
    HOME = "home"
    MIDGAME = "midgame"
    QUICK_PLAY = "quick_play"
    LEVEL_SELECT = "level_select"
    ARCADE = "arcade"
    GREAT_PLAY = "great_play"

TILE_INFO = {
    Tile.ANCHOR_BOX: {"emoji": "📦", "color": 26},
    Tile.BACKGROUND: {"emoji": "　", "color": 236},
    Tile.BED: {"emoji": "🛌", "color": 37},
    Tile.BENCH: {"emoji": "🪑", "color": 38},
    Tile.BLANK: {"emoji": "　", "color": 238},
    Tile.BLENDER: {"emoji": "🍶", "color": 39},
    Tile.BLUE_DRAGON: {"emoji": "🐲", "color": 49},
    Tile.BLUE_FISH: {"emoji": "🌑", "color": 19},
    Tile.CHAIR: {"emoji": "💺", "color": 33},
    Tile.CHEST: {"emoji": "🌰", "color": 105},
    Tile.CLOWNFISH: {"emoji": "🤡", "color": 208},
    Tile.COIN: {"emoji": "🪙", "color": 247},
    Tile.COLUMN: {"emoji": "🏛", "color": 87},
    Tile.CRAB: {"emoji": "🦀", "color": 196},
    Tile.FOSSIL: {"emoji": "🦴", "color": 180},
    Tile.FRIDGE: {"emoji": "🧊", "color": 32},
    Tile.GOLDFISH: {"emoji": "🥇", "color": 220},
    Tile.GREEN_SEAHORSE: {"emoji": "🐴", "color": 47},
    Tile.LIONFISH: {"emoji": "🦁", "color": 172},
    Tile.LOCKED: {"emoji": "🔒", "color": 0}, #136
    Tile.MAGENTA_FISH: {"emoji": "🍣", "color": 162},
    Tile.ORANGE_FISH: {"emoji": "🍤", "color": 210},
    Tile.OVEN: {"emoji": "🎛", "color": 27},
    Tile.PINK_FISH: {"emoji": "🍥", "color": 218},
    Tile.PURPLE_DRAGON: {"emoji": "🐲", "color": 177},
    Tile.PURPLE_FISH: {"emoji": "🍇", "color": 92},
    Tile.ROUND_VASE: {"emoji": "🥣", "color": 179},
    Tile.SEAL: {"emoji": "🦭", "color": 250},
    Tile.SHELL: {"emoji": "🐚", "color": 221},
    Tile.STARFISH: {"emoji": "⭐️", "color": 209},
    Tile.THRONE: {"emoji": "👑", "color": 45},
    Tile.TRIANGLE_VASE: {"emoji": "🏺", "color": 130},
    Tile.TURTLE: {"emoji": "🐢", "color": 28},
    Tile.UMBRELLA: {"emoji": "⛱", "color": 44},
    Tile.WHALE: {"emoji": "🐋", "color": 242},
    Tile.WHEEL_TABLE: {"emoji": "🎡", "color": 180},
    Tile.YELLOW_FISH: {"emoji": "🌻", "color": 222},
    Tile.YELLOW_SEAHORSE: {"emoji": "🐴", "color": 223},
}

DIR_OFFSETS = {
    Dir.LEFT: (-1,0),
    Dir.RIGHT: (1,0),
    Dir.UP: (0,-1),
    Dir.DOWN: (0,1)
}

SCREEN_TYPES = {
    Screen.BONUS: [700, 515, [120, 77, 23]],
    Screen.OUT_OF_MOVES: [805, 480, [116, 20, 13]],
    Screen.GAME_OVER: [30, 1170, [137, 81, 203]],
    Screen.CONGRATS: [80, 80, [254, 253, 86]],
    Screen.HOME: [650, 1150, [240, 145, 53]],
    Screen.MIDGAME: [1210, 60, [164, 246, 77]],
    Screen.QUICK_PLAY: [477, 105, [239, 194, 96]],
    Screen.LEVEL_SELECT: [1090, 750, [0, 2, 57]],
    Screen.ARCADE: [390, 1010, [145, 61, 149]],
    Screen.GREAT_PLAY: [700, 1165, [239, 140, 52]]
}

LEVEL_INFO = {
    0: {"pairs": 64, "time": 500},
    1: {"pairs": 38, "time": 200}, # got a 25s
    2: {"pairs": 48, "time": 350}, # got in 32s
    3: {"pairs": 49, "time": 575}, # got in 32s
    4: {"pairs": 52, "time": 550},
    5: {"pairs": 56, "time": 880},
    6: {"pairs": 54, "time": 980},
    7: {"pairs": 52, "time": 1450},
    8: {"pairs": 46,  "time": 1800}
}

def get_board_dims(level):
    tile_w, tile_h = 94, 101
    board_x, board_y = 55, 180
    tile_m, tile_n = 16, 8 # width, height
    if level == 1:
        tile_m, tile_n = 14, 6
        board_x += tile_w
        board_y += tile_h+46
    elif level == 5:
        tile_m, tile_n = 14, 8
        board_x += tile_w
    board_w, board_h = tile_w*tile_m, tile_h*tile_n
    return tile_w, tile_h, board_x, board_y, tile_m, tile_n, board_w, board_h

def capture_board(level):
    _, _, board_x, board_y, _, _, board_w, board_h = get_board_dims(level)
    return io.capture_portion(board_x, board_y, board_w, board_h)

def get_tile_imgs_from_board(level, board_img):
    tile_w, tile_h, _, _, tile_m, tile_n, _, _ = get_board_dims(level)
    tile_images = board_img.reshape(tile_m, tile_w, tile_n, tile_h, 3)
    return np.transpose(tile_images, (0, 2, 1, 3, 4))

def get_cropped_center_of_image(image, w=10):
    width, height, _ = image.shape
    return image[width//2-w//2:width//2+w//2,height//2-w//2:height//2+w//2]

def click_hint_btn(delays=None):
    io.click_mouse(1450, 1110, delays)

def click_tile(level, i, j, delays=None):
    tile_w, tile_h, board_x, board_y, _, _, _, _ = get_board_dims(level)
    x, y = tile_w*(i+0.5)+board_x, tile_h*(j+0.5)+board_y
    io.click_mouse(x, y, delays)
    

# Given a board image, save the tiles named with their avg. color, for later organization
def save_tile_imgs(level, board_img):
    tile_w, tile_h, _, _, tile_m, tile_n, _, _ = get_board_dims(level)
    rgb2hex = lambda r,g,b: '%02X%02X%02X' % (int(r),int(g),int(b))
    for i in range(tile_m):
        for j in range(tile_n):
            tile_x, tile_y = i*tile_w, j*tile_h
            tile = board_img[tile_x:tile_x+tile_w,tile_y:tile_y+tile_h]
            avg_pixel = np.mean(tile, axis=(0,1))
            avg_pixel_hex = rgb2hex(*avg_pixel)
            filename = f"tiles/{avg_pixel_hex}_{i}_{j}_5.png"
            io.save_image(tile, filename)

def get_most_changed_tiles(level, board_img_after, board_img_before, ignore_indices=[], n=2):
    board_diff = np.absolute(np.subtract(board_img_after, board_img_before))
    changed_pixels = (board_diff != [0, 0, 0]).all(axis=2)
    black_pixels = np.zeros_like(board_diff)
    white_pixels = np.ones_like(board_diff) * 255
    board_mask = np.where(changed_pixels[:,:,None], white_pixels, black_pixels).astype(np.uint8)
    tile_images = get_tile_imgs_from_board(level, board_mask)
    num_changed_pixels = np.sum(tile_images, axis=(2,3,4))
    extra = len(ignore_indices)
    flat_sorted_indices = np.argsort(num_changed_pixels.flatten())[-(n+extra):][::-1]
    x_indices, y_indices = np.unravel_index(flat_sorted_indices, num_changed_pixels.shape)
    indices = list(zip(x_indices, y_indices))
    for i, ignore_idx in enumerate(ignore_indices):
        if ignore_idx in indices:
            indices.remove(ignore_idx)
    assert len(indices) >= n, "Can't find a non-ignored pair"
    return indices[:n]

# Repeatedly clicks the 'Hint' button and the two tiles that are highlighted. Pretty naive, sometimes misclicks.
def solve_level_with_hints(level):
  time.sleep(2)
  last_highlighted_tiles = []
  CLICK_BUFFER = 4
  num_pairs = LEVEL_INFO[level]["pairs"]
  for i in range(num_pairs):
    board_img_before = capture_board(level)
    click_hint_btn(delays=[0.05,0.05,0])
    time.sleep(0.05)
    board_img_after = capture_board(level)

    highlighted_tiles = get_most_changed_tiles(level, board_img_after, board_img_before, last_highlighted_tiles)
    last_highlighted_tiles = last_highlighted_tiles[-CLICK_BUFFER:] + highlighted_tiles

    for i, j in highlighted_tiles:
      click_tile(level, i, j, delays=[0.05,0.05,0])
  
# This gets the tile images saved previously and creates an average tile for each type which we can later use for masks
def gen_save_average_tile_pngs(main_folder = 'tiles', save_path='_average'):
  for subdir in os.listdir(main_folder):
      subdir_path = os.path.join(main_folder, subdir)
      if subdir[0] in ['.', '_'] or not os.path.isdir(subdir_path) or save_path in subdir_path:
          continue
      image_names = [f for f in os.listdir(subdir_path) if f.endswith('.png')]
      image_list = [io.load_image(os.path.join(subdir_path, image_name)) for image_name in image_names]
      average_img = IO.get_average_image(image_list)
      average_image_path = os.path.join(main_folder, f'{save_path}/{subdir}.png')
      io.save_image(average_img, average_image_path)

# Makes sure that the avg_tiles images can correctly predict a tile type
def test_avg_tile_mask_proficiency(masks, w=None, main_folder = 'tiles'):
    if w is not None:
        masks = {mask_name: get_cropped_center_of_image(mask,w=w) for mask_name, mask in masks.items()}
    for subdir in os.listdir(main_folder):
        subdir_path = os.path.join(main_folder, subdir)
        if subdir[0] in ['.', '_'] or not os.path.isdir(subdir_path):
            continue
        image_files = [f for f in os.listdir(subdir_path) if f.endswith('.png')]
        for image_file in image_files:
            image_path = os.path.join(subdir_path, image_file)
            image = io.load_image(image_path)
            if w is not None:
                image = get_cropped_center_of_image(image, w=w)
            idx, _ = IO.(image, masks.values())
            best_img_name = list(masks.items())[idx][0]
            if subdir not in best_img_name:
                raise Exception("ISSUE: not best match", best_img_name, image_file, subdir)

def tile_type_from_name(tile_name):
    enum_member = next((member for member in Tile.__members__.values() if member.value == tile_name), None)
    if enum_member is not None:
        return enum_member
    raise Exception("No such enum", tile_name)

def get_best_tile_type(tile_img, tile_masks, w=None):
    if w is not None:
        tile_masks = {mask_name: get_cropped_center_of_image(mask,w=w) for mask_name, mask in tile_masks.items()}
        tile_img = get_cropped_center_of_image(tile_img, w=w)
    idx, _ = IO.get_best_mask(tile_img, tile_masks.values())
    tile_name = list(tile_masks.items())[idx][0][:-4]
    return tile_type_from_name(tile_name)

def get_tile_str(tile_enum, p=1):
    code = TILE_INFO[tile_enum]['color']
    emoji = TILE_INFO[tile_enum]['emoji']
    return u"\u001b[48;5;" + str(code) + "m" + " "*p + f"{emoji}" + " " + u"\u001b[0m"

def print_board(board_tiles, p=1):
    M, N = len(board_tiles), len(board_tiles[0])
    output = []
    output.append((" "*p +"　").join([f"{x:>2}" for x in [''] + list(range(N))]))
    for i, row in enumerate(board_tiles):
        line = f"{i:>2} "
        for tile in row:
            line += get_tile_str(tile, p=p) + " "
        output.append(f"{line}")
    output = '\n'.join(output)
    print(output)
    return output

def _find_matches_for_single_tile(board_tiles, path_coords, target_dests, turns_left, last_dir, ):
    board_tiles_imm = tuple(map(tuple, board_tiles))
    path_coords_imm = tuple(path_coords)
    target_dests_imm = tuple(target_dests)
    return _find_matches_for_single_tile_cached(board_tiles_imm, path_coords_imm, target_dests_imm, turns_left, last_dir)

@lru_cache(maxsize=512)
def _find_matches_for_single_tile_cached(board_tiles, path_coords, target_dests, turns_left, last_dir):
    M, N = len(board_tiles), len(board_tiles[0])
    start_pos = path_coords[0]
    si, sj = start_pos
    target_tile_type = board_tiles[si][sj]
    last_pos = path_coords[-1]
    target_dests = set(target_dests)
    solution_paths = []
    for new_direction, dir_offset in DIR_OFFSETS.items():
        new_pos = (last_pos[0] + dir_offset[0], last_pos[1] + dir_offset[1])
        ni, nj = new_pos
        # Ignore this tile if we've already visited it (in path)
        if new_pos in path_coords: # TODO: set for better Big-O?
            continue
        # Ignore this direction it goes out of bounds
        if not (0 <= ni < M and 0 <= nj < N):
            continue
        new_turns_left = turns_left
        # If we are changing directions, spend a turn (if able)
        if last_dir is not None and last_dir != new_direction:
            # If we wanted to turn but have no more, abandon direction
            if turns_left == 0:
                continue
            new_turns_left -= 1
        # Add this position to the path
        new_path_coords = list(path_coords) + [new_pos]
        # Check if we are continuing the path, hit a wall, or found a pair
        new_tile_type = board_tiles[ni][nj]
        # TODO: IMPORTANT it finds a lot of duplicate paths especially on a wide-open board
        #       should prob instead to try to find *a* path to each match, could use A* w/ multiple goals
        # If we hit a match, return the path
        if new_tile_type == target_tile_type:
            solution_paths.append(new_path_coords)
            if new_pos in target_dests:
                target_dests.remove(new_pos)
            if len(target_dests) == 0:
                return solution_paths
            continue
        # If we hit a wall (locked or non-matching tile), abandon direction
        elif new_tile_type not in [Tile.BLANK, Tile.BACKGROUND]:
            continue
        # If we are continuing on an open path, recurse
        else:
            new_solution_paths = _find_matches_for_single_tile(board_tiles, new_path_coords, target_dests, new_turns_left, new_direction)
            solution_paths += new_solution_paths
            for solution_path in new_solution_paths:
                dest_pos = solution_path[-1]
                if dest_pos in target_dests:
                    target_dests.remove(dest_pos)
                if len(target_dests) == 0:
                    return solution_paths
    return solution_paths

def find_matches_for_single_tile(board_tiles, start_pos):
    i, j = start_pos
    path_coords = [start_pos]
    target_tile_type = board_tiles[i][j]
    # No solutions for a blank, background, or locked tile type
    if target_tile_type in [Tile.LOCKED, Tile.BLANK, Tile.BACKGROUND]:
        return []
    tile_distrib = get_tile_distrib(board_tiles)
    target_dests = tile_distrib[target_tile_type].copy()
    target_dests.remove(start_pos)
    solution_paths = _find_matches_for_single_tile(board_tiles, path_coords, target_dests, turns_left=2, last_dir=None)
    unique_matches = set([path[-1] for path in solution_paths])
    return solution_paths, unique_matches

def find_all_matches(board_tiles):
    board_tiles_imm = tuple(map(tuple, board_tiles))
    return _find_all_matches_cached(board_tiles_imm)

@lru_cache(maxsize=512)
def _find_all_matches_cached(board_tiles):
    all_pairs = defaultdict(set)
    M, N = len(board_tiles), len(board_tiles[0])
    for i in range(M):
        for j in range(N):
            start_pos = i,j
            tile_type = board_tiles[i][j]
            if tile_type in [Tile.BACKGROUND, Tile.BLANK, Tile.LOCKED]:
                continue
            solutions, unique_matches = find_matches_for_single_tile(board_tiles, (i,j))
            ordered_pairs = [(start_pos, other_pos) if start_pos < other_pos else (other_pos, start_pos) for other_pos in unique_matches]
            all_pairs[tile_type].update(ordered_pairs)
    return all_pairs

def get_tile_distrib(board_tiles):
    board_tiles_imm = tuple(map(tuple, board_tiles))
    return _get_tile_distrib_cached(board_tiles_imm)

@lru_cache(maxsize=128)
def _get_tile_distrib_cached(board_tiles):
    M, N = len(board_tiles), len(board_tiles[0])
    tile_distrib = defaultdict(set)
    for i in range(M):
        for j in range(N):
            tile_type = board_tiles[i][j]
            tile_distrib[tile_type].add((i,j))
    return tile_distrib

def is_board_empty(board_tiles):
    flat_board = [tile for row in board_tiles for tile in row]
    unique_tiles = set(flat_board)
    return unique_tiles.issubset(set([Tile.BLANK, Tile.BACKGROUND, Tile.LOCKED]))

# Remove the pair (replace with blanks)
def remove_pair_inplace(board_tiles, pair):
    tile_start, tile_end = pair
    ti, tj = tile_start
    board_tiles[ti][tj] = Tile.BLANK
    ti, tj = tile_end
    board_tiles[ti][tj] = Tile.BLANK

# Calc distance between pairs to use for sorting
def calc_distance(pair):
    coord_1, coord_2 = pair
    i1, j1 = coord_1
    i2, j2 = coord_2
    return abs(i1-i2) + abs(j1-j2)

def calc_freed_neighbors(board_tiles, pair):
    coord_1, coord_2 = pair
    neighbor_coords = set()
    for dir_offset in DIR_OFFSETS.values():
        di, dj = dir_offset
        ci, cj = coord_1
        neighbor_coords.add((ci + di, cj + dj))
        ci, cj = coord_2
        neighbor_coords.add((ci + di, cj + dj))
    neighbor_coords.difference_update(pair)
    freed_neighbors = set()
    for neighbor_coord in neighbor_coords:
        ni, nj = neighbor_coord
        neighbor_type = board_tiles[ni][nj]
        if neighbor_type not in [Tile.BLANK, Tile.BACKGROUND, Tile.LOCKED]:
            freed_neighbors.add(neighbor_coord)
    return freed_neighbors

# TODO: certain levels seem unsolveable
def find_a_solution(board_tiles):
    board_tiles_imm = tuple(map(tuple, board_tiles))
    return _find_a_solution_cached(board_tiles_imm)

@lru_cache(maxsize=512)
def _find_a_solution_cached(board_tiles):
    # If board is empty, we've succeeded
    if is_board_empty(board_tiles):
        return []
    all_pairs = find_all_matches(board_tiles)
    flat_pairs = set().union(*all_pairs.values())
    tile_distrib = get_tile_distrib(board_tiles)
    # Find the types which only have one pair left
    one_pair_types = set([tile_type for tile_type, tiles in tile_distrib.items() if len(tiles) == 2])
    # Find the existing pair of that type (if one exists)
    gimme_pairs = set([pair for tile_type, pairs in all_pairs.items() for pair in pairs if tile_type in one_pair_types])
    # If there are gimme pairs, use one of them (if it fails, all must fail)
    if len(gimme_pairs) > 0:
        existing_pairs = {next(iter(gimme_pairs))}
    else:
        existing_pairs = sorted(flat_pairs, key=lambda x: (calc_distance(x), -len(calc_freed_neighbors(board_tiles, x)))) # Can't believe this worked!
    # If no pairs found (and board not empty), we can't find a solution
    if len(existing_pairs) == 0:
        return None
    for next_pair in existing_pairs:
        new_board_tiles = [list(row) for row in board_tiles]
        remove_pair_inplace(new_board_tiles, next_pair)
        new_solution_pairs = find_a_solution(new_board_tiles)
        # If this pair failed, try the next one
        if new_solution_pairs is None:
            continue
        # If we found a solution for the remaining board, prepend this pair & return
        return [next_pair] + new_solution_pairs
    # We weren't able to find any valid next pair (they all ran OOM)
    return None

def get_solution_str(tile_enum, idx, p=1):
    code = TILE_INFO[tile_enum]['color']
    return u"\u001b[48;5;" + f"{code}m" + "\u001b[1m" + " "*p +f"{idx:^2}" + " "*p + u"\u001b[0m"

def print_solution(board_tiles, solution, p=1):
    M, N = len(board_tiles), len(board_tiles[0])
    board_tiles = [list(row) for row in board_tiles]
    tile_indices = defaultdict(lambda: "")
    for idx, pair in enumerate(solution):
        coord_1, coord_2 = pair
        tile_indices[coord_1] = idx
        tile_indices[coord_2] = idx
    output = []
    output.append((" "*(p*2+1)).join([f"{x:>2}" for x in [''] + list(range(N))]))
    for i, row in enumerate(board_tiles):
        line = f"{i:>2} "
        for j, tile in enumerate(row):
            tile_enum = board_tiles[i][j]
            tile_idx = tile_indices[(i,j)]
            line += get_solution_str(tile_enum, tile_idx, p=p) + " "
        output.append(f"{line}")
    output = '\n'.join(output)
    print(output)
    return output

def parse_board_tiles_from_image(level, game_img, tile_masks, w=None):
    _, _, board_x, board_y, tile_m, tile_n, board_w, board_h = get_board_dims(level)
    board_img = IO.crop_portion(board_x, board_y, board_w, board_h, game_img)
    tile_images = get_tile_imgs_from_board(level, board_img)
    board_tiles = [[Tile.BACKGROUND]*(tile_m+2)]
    for y in range(tile_n):
        row = [Tile.BACKGROUND]
        for x in range(tile_m):
            tile_img = tile_images[x,y]
            tile_type = get_best_tile_type(tile_img, tile_masks, w=w)
            row.append(tile_type)
        row.append(Tile.BACKGROUND)
        board_tiles.append(row)
    board_tiles.append([Tile.BACKGROUND]*(tile_m+2))
    return board_tiles

def is_screen_type(screen_type, image=None):
    x, y, color = SCREEN_TYPES[screen_type]
    return io.is_given_screen(x, y, color, delta=10, image=image)

def calc_screen_types(image=None):
    return {screen_type for screen_type in SCREEN_TYPES.keys() if is_screen_type(screen_type, image)}

def crop_level_number(image=None):
    x, y ,w ,h = 564, 58, 34, 44
    if image is None:
        return io.capture_portion(x, y, w, h)
    return io.crop_portion(x, y, w, h, image)

def calc_level(game_img, level_masks):
    level_img = crop_level_number(game_img)
    level_idx, _ = IO.get_best_mask(level_img, level_masks.values())
    curr_level = int(list(level_masks.keys())[level_idx][:-4])
    is_level = is_screen_type(Screen.MIDGAME, game_img)
    if is_level is None:
        return None
    return curr_level

def save_game_cap(level, game_img=None):
    if game_img is None:
        game_img = io.capture_screen()
    now = datetime.datetime.now()
    date_str = now.strftime("%Y-%m-%d %H-%M-%S")
    io.save_image(game_img, f"no-commit/saved/level-{level} {date_str}.png")

# TODO: solution pairs are screwed around to be (j,i)/(y,x) and M,N confused.
def submit_solution(level, solution):
    io.click_mouse(0, 0) # Refocus
    for pair in solution:
        coord_1, coord_2 = pair
        # Game doesn't recognize certain diagonal pairs, try both ways of entering
        i, j = coord_1
        click_tile(level, j-1, i-1, delays=[0.05,0.05,0])
        i, j = coord_2
        click_tile(level, j-1, i-1, delays=[0.05,0.05,0])
        i, j = coord_1
        click_tile(level, j-1, i-1, delays=[0.05,0.05,0])

def get_into_game(game_mode = 'level'):
  global kill_program
  # Get to the main play screen
  while True:
    if kill_program:
      break
    game_img = io.capture_screen()
    screen_types = calc_screen_types(game_img)
    # print(screen_types)

    if Screen.HOME in screen_types:
      io.click_mouse(810, 980)
    elif Screen.LEVEL_SELECT in screen_types:
      if game_mode == 'level':
        io.click_mouse(1100, 810) # "Level Mode" button
      elif game_mode == 'quick':
        io.click_mouse(500, 800) # "Quick Play" button
    elif Screen.GAME_OVER in screen_types:
      io.click_mouse(530, 1000) # "Replay" button
    elif Screen.CONGRATS in screen_types:
      io.click_mouse(1470, 70) # "X" button
    elif Screen.GREAT_PLAY in screen_types:
      io.click_mouse(800, 1100) # "Collect" button
    elif Screen.MIDGAME in screen_types:
      if Screen.OUT_OF_MOVES in screen_types:
        io.click_mouse(1470, 70) # "X" button
      elif Screen.BONUS in screen_types:
        time.sleep(5) # Wait for new level
      else:
        return
    time.sleep(0.5)

# TODO: move to utils?
QUIT_KEY = '`'
def on_press(key):
  global kill_program, listener
  if key == kb.KeyCode(char=QUIT_KEY):
    print(f"{key} pressed to quit")
    listener.stop()
    kill_program = True

In [ ]:
kill_program = False

listener = kb.Listener(on_press=on_press)
listener.start()
# Load tile & level # masks
tile_masks = io.load_images_from_directory('tiles/_average')
level_masks = io.load_images_from_directory("levels")

# Run game loop
while True:
  if kill_program:
      break
  io.click_mouse(0, 0) # Focus Webkinz
  get_into_game('level')

  game_img = io.capture_screen()
  level = calc_level(game_img, level_masks)
  print(f"\n===== LEVEL {level} =====")
  # save_game_cap(level, game_img)
  
  board_tiles = parse_board_tiles_from_image(level, game_img, tile_masks, w=90)
  print("BOARD:")
  print([[x.value for x in row] for row in board_tiles])
  print_board(board_tiles, p=0)

  start = time.time()
  solution = find_a_solution(board_tiles)
  end = time.time()
  print("SOLUTION:")
  print(solution)
  print_solution(board_tiles, solution, p=0)
  print(f"SOLVE TIME: {end - start: 0.2f} seconds")

  submit_solution(level, solution)
  time.sleep(2)
  
listener.stop()